## import libraries

In [52]:
from compas.geometry import Frame, Plane, Polyhedron, Vector
from compas.geometry import Box, Sphere
from compas_vol.primitives import VolBox, VolSphere
from compas_vol.combinations import Union, SmoothUnion, Intersection, SmoothUnionList
from compas_vol.modifications import Shell

In [2]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import get_random_vector_3D, bbox_edges

## create volumetric object (CSG tree)

In [53]:
# generate points on a sphere for the edges meeting in the node
polyh = Polyhedron.from_platonicsolid(20)
pts = polyh.vertices

In [54]:
#create volumetric boxes and unify them
sphereRad = 3.4
volBoxes = []
for [x,y,z] in pts:
    v = Vector(x,y,z)
    v.unitize()
    v *= sphereRad
    plane = Plane([v.x, v.y, v.z],[v.x, v.y, v.z])
    box = Box(Frame.from_plane(plane),1.2,1.2,3.5)
    vbox = VolBox(box, 0.1)
    volBoxes.append(vbox)
uvb = Union(volBoxes)

In [105]:
#combine core and branches
vs = VolSphere(Sphere([0,0,0], 2.5))
#su = SmoothUnion(uvb, vs, 3.0)
su = SmoothUnionList([uvb, vs], 5.0)

In [106]:
#thicken shell and cull extremities
s = Shell(su, 0.2, 1.0)
node = Intersection(VolSphere(Sphere([0,0,0], 4.8)), s)

## workspace (dense grid)

In [107]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -5.0, 5.0
lby, uby = -5.0, 5.0
lbz, ubz = -5.0, 5.0
# resolution(s)
nx, ny, nz = 120, 120, 120
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

## sample at discrete interval

In [108]:
dm = node.get_distance_numpy(x, y, z)

## generate isosurface (marching cube)

In [109]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

## display mesh

In [110]:
p = mp.plot(v, f, c=np.array([0,0.57,0.82]), shading={"flat":False, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})
vs,ve = bbox_edges(lbx,ubx,lby,uby,lbz,ubz)
p.add_lines(np.array(vs), np.array(ve))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.041666…

1

In [66]:
import matplotlib.pyplot as plt
from ipywidgets import interactive

In [ ]:
#dm[dm>0] /= dm.max()
#dm[dm<0] /= -dm.min()

In [113]:
def slice(z=0):
    plt.figure(figsize=(10,10))
    plt.imshow(dm[:,:,z], cmap='RdBu', extent=[lbx,ubx,uby,lby])

In [114]:
interactive(slice, z=(0,nz-1,1))

interactive(children=(IntSlider(value=0, description='z', max=119), Output()), _dom_classes=('widget-interact'…